# Info Extraction

it's much more easier to extract information of model from pytorch module than onnx...onnx doesn't have output shape

In [1]:
import onnx

# Load the ONNX model
model = onnx.load("onnx/vgg19.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %1[FLOAT, 1x3x224x224]
) initializers (
  %2[FLOAT, 64x3x3x3]
  %3[FLOAT, 64]
  %4[FLOAT, 64x64x3x3]
  %5[FLOAT, 64]
  %6[FLOAT, 128x64x3x3]
  %7[FLOAT, 128]
  %8[FLOAT, 128x128x3x3]
  %9[FLOAT, 128]
  %10[FLOAT, 256x128x3x3]
  %11[FLOAT, 256]
  %12[FLOAT, 256x256x3x3]
  %13[FLOAT, 256]
  %14[FLOAT, 256x256x3x3]
  %15[FLOAT, 256]
  %16[FLOAT, 256x256x3x3]
  %17[FLOAT, 256]
  %18[FLOAT, 512x256x3x3]
  %19[FLOAT, 512]
  %20[FLOAT, 512x512x3x3]
  %21[FLOAT, 512]
  %22[FLOAT, 512x512x3x3]
  %23[FLOAT, 512]
  %24[FLOAT, 512x512x3x3]
  %25[FLOAT, 512]
  %26[FLOAT, 512x512x3x3]
  %27[FLOAT, 512]
  %28[FLOAT, 512x512x3x3]
  %29[FLOAT, 512]
  %30[FLOAT, 512x512x3x3]
  %31[FLOAT, 512]
  %32[FLOAT, 512x512x3x3]
  %33[FLOAT, 512]
  %34[FLOAT, 4096x25088]
  %35[FLOAT, 4096]
  %36[FLOAT, 4096x4096]
  %37[FLOAT, 4096]
  %38[FLOAT, 1000x4096]
  %39[FLOAT, 1000]
) {
  %41 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%1, %

In [2]:
#import onnx_caffe2.backend as backend
import onnx_tf.backend as backend
import numpy as np
import time

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Extract Shape of Initial Input Object

In [ ]:
%%time
rep = backend.prepare(model, device="CUDA:0") # or "CPU"

In [ ]:
rep.input_dict['2']

In [ ]:
def get_init_shape_dict(rep):
    d = {}
    for key in rep.input_dict:
        tensor = rep.input_dict[key]
        shape = np.array(tensor.shape, dtype=int)
        d.update({key:shape})
    return d
shape_dict = get_init_shape_dict(rep)

In [ ]:
len(shape_dict)

In [ ]:
import pickle
pickle.dump(shape_dict,open('onnx/vgg19_init_shape_dict.pkl','wb'))

## Extract Shape of Outputed Object

In [3]:
import pickle
init_shape_dict = pickle.load(open('onnx/vgg19_init_shape_dict.pkl','rb'))

In [4]:
len(init_shape_dict)

39

In [5]:
def get_output_shape_of_node(node, shape_dict, device = "CPU"):# or "CUDA:0"
    
    out_idx = node.output[0]
    input_list = node.input # e.g. ['1', '2']
    
    inps = []
    for inp_idx in input_list:
        inp_shape = shape_dict[inp_idx] 
        rand_inp = np.random.random(size=inp_shape).astype('float16')
        inps.append(rand_inp)
    try:
        out = backend.run_node(node=node, inputs=inps, device=device)
        out_shape = out[0].shape 
    except:
        out_shape = shape_dict[input_list[0]]
        print("Op: [{}] run_node error! return inp_shape as out_shape".format(node.op_type))
        
    return out_shape, out_idx 

In [ ]:
# test 
get_output_shape_of_node(model.graph.node[0], init_shape_dict.copy())

In [ ]:
# test Op: [Dropout]
shape_dict1 = init_shape_dict.copy()
shape_dict1.update({'120':[1,3,4,5]})
get_output_shape_of_node(model.graph.node[59], shape_dict1)

In [ ]:
# test Op: [Gemm]
shape_dict1 = init_shape_dict.copy()
shape_dict1.update({'122':[1,4096]})
for i, node in enumerate(model.graph.node[60:]):
    st=time.time()
    out_shape, out_idx = get_output_shape_of_node(node, shape_dict1)
    shape_dict1.update({out_idx:out_shape})
    print("out_shape: {} for Obj[{}], node [{}][{}]...{:.2f} sec".format(out_shape, out_idx, i, node.op_type,time.time()-st))

In [7]:
# takes time!!!
# 雖然可以自己算，怕網路複雜時會出錯，故還是用onnx backend去跑出output_shape(會多花點時間)
def get_overall_shape_dict(init_shape_dict):
    shape_dict = init_shape_dict.copy()
    for i, node in enumerate(model.graph.node):
        st=time.time()
        out_shape, out_idx = get_output_shape_of_node(node, shape_dict)
        shape_dict.update({out_idx:out_shape})
        print("out_shape: {} for Obj[{}], node [{}][{}]...{:.2f} sec".format(out_shape, out_idx, i, node.op_type,time.time()-st))
    return shape_dict

In [8]:
overall_shape_dict = get_overall_shape_dict(init_shape_dict)   

/usr/local/lib/python2.7/dist-packages/onnx_tf/backend.py:677: UserWarning: Unsupported kernel_shape attribute by Tensorflow in Conv operator. The attribute will be ignored.
  UserWarning)


out_shape: (1, 64, 224, 224) for Obj[41], node [0][Conv]...0.43 sec
out_shape: (1, 64, 224, 224) for Obj[42], node [1][Add]...7.51 sec
out_shape: (1, 64, 224, 224) for Obj[43], node [2][Relu]...7.49 sec
out_shape: (1, 64, 224, 224) for Obj[45], node [3][Conv]...8.04 sec
out_shape: (1, 64, 224, 224) for Obj[46], node [4][Add]...8.25 sec
out_shape: (1, 64, 224, 224) for Obj[47], node [5][Relu]...8.45 sec
out_shape: (1, 64, 112, 112) for Obj[48], node [6][MaxPool]...9.18 sec
out_shape: (1, 128, 112, 112) for Obj[50], node [7][Conv]...4.09 sec
out_shape: (1, 128, 112, 112) for Obj[51], node [8][Add]...5.81 sec
out_shape: (1, 128, 112, 112) for Obj[52], node [9][Relu]...6.03 sec
out_shape: (1, 128, 112, 112) for Obj[54], node [10][Conv]...6.48 sec
out_shape: (1, 128, 112, 112) for Obj[55], node [11][Add]...6.49 sec
out_shape: (1, 128, 112, 112) for Obj[56], node [12][Relu]...6.56 sec
out_shape: (1, 128, 56, 56) for Obj[57], node [13][MaxPool]...6.60 sec
out_shape: (1, 256, 56, 56) for Obj[5

In [9]:
len(overall_shape_dict)

100

In [10]:
pickle.dump(overall_shape_dict,open('onnx/vgg19_overall_shape_dict.pkl','wb'))

## Get Input/Output/Kernel Shape for Conv

In [11]:
import pickle
overall_shape_dict = pickle.load(open('onnx/vgg19_overall_shape_dict.pkl','rb'))

In [19]:
def get_kernel_shape_dict():
    conv_d = {}
    for i, node in enumerate(model.graph.node):
        if node.op_type == 'Conv':
            for attr in node.attribute:
                if attr.name == "kernel_shape":
                    kernel_shape = np.array(attr.ints, dtype=int)
                    break
            inp_idx = node.input[0]
            out_idx = node.output[0]
            inp_shape = overall_shape_dict[inp_idx]
            out_shape = overall_shape_dict[out_idx]
            conv_d.update({i:(inp_idx, out_idx, inp_shape, out_shape, kernel_shape)})
            print("for node [{}][{}]:\ninp_shape: {} from obj[{}], \nout_shape: {} from obj[{}], \nkernel_shape: {} \n"
                  .format(i, node.op_type, inp_shape, inp_idx, out_shape, out_idx, kernel_shape ))
    return conv_d

In [20]:
conv_d = get_kernel_shape_dict()

for node [0][Conv]:
inp_shape: [  1   3 224 224] from obj[1], 
out_shape: (1, 64, 224, 224) from obj[41], 
kernel_shape: [3 3] 

for node [3][Conv]:
inp_shape: (1, 64, 224, 224) from obj[43], 
out_shape: (1, 64, 224, 224) from obj[45], 
kernel_shape: [3 3] 

for node [7][Conv]:
inp_shape: (1, 64, 112, 112) from obj[48], 
out_shape: (1, 128, 112, 112) from obj[50], 
kernel_shape: [3 3] 

for node [10][Conv]:
inp_shape: (1, 128, 112, 112) from obj[52], 
out_shape: (1, 128, 112, 112) from obj[54], 
kernel_shape: [3 3] 

for node [14][Conv]:
inp_shape: (1, 128, 56, 56) from obj[57], 
out_shape: (1, 256, 56, 56) from obj[59], 
kernel_shape: [3 3] 

for node [17][Conv]:
inp_shape: (1, 256, 56, 56) from obj[61], 
out_shape: (1, 256, 56, 56) from obj[63], 
kernel_shape: [3 3] 

for node [20][Conv]:
inp_shape: (1, 256, 56, 56) from obj[65], 
out_shape: (1, 256, 56, 56) from obj[67], 
kernel_shape: [3 3] 

for node [23][Conv]:
inp_shape: (1, 256, 56, 56) from obj[69], 
out_shape: (1, 256, 56, 56

## Calculate n_param, n_flops

In [22]:
conv_d = pickle.load(open('onnx/vgg19_conv_shape_dict.pkl','rb'))

In [23]:
n_param = 0
n_flops = 0
for k in conv_d:
    #i:(inp_idx, out_idx, inp_shape, out_shape, kernel_shape)
    inp_shape, out_shape, kernel_shape = conv_d[k][2],conv_d[k][3],conv_d[k][4]
    h,w,c,n,H,W = kernel_shape[1], kernel_shape[1], inp_shape[1], out_shape[1], out_shape[2], out_shape[3]
    n_param  += n*(h*w*c+1)
    n_flops  += H*W*n*(h*w*c+1)
n_param, n_flops

(20024384, 19523280896)